In [4]:
import pandas as pd
import numpy as np
import logging
from math import floor
from google.oauth2.service_account import Credentials
import gspread
from typing import Dict, Any, List
from utils import get_teg_rounds, load_all_data, aggregate_data





In [ ]:
all_data = load_all_data(exclude_teg_50=True,exclude_incomplete_tegs=True)
print(all_data)

In [ ]:
all_data = load_all_data(exclude_teg_50=True,exclude_incomplete_tegs=True)
agg_measures = ['Sc','GrossVP','NetVP','Stableford']
teg_data = aggregate_data(all_data,'TEG',agg_measures)

teg_data

In [ ]:
df = teg_data

best_stableford = df.loc[df.groupby('Player')['Stableford'].idxmax()]

# Select only relevant columns for output
output = best_stableford[['Player', 'Stableford', 'TEG']].rename(columns={'Stableford': 'Best Stableford'})

# Display output
print(output)

In [ ]:

# Find unique combinations of TEG and Year
unique_teg_year = all_data[['TEG', 'Year']].drop_duplicates()

# Merge teg_data with the unique TEG-Year combinations to add the Year column
df_merged = pd.merge(teg_data, unique_teg_year, on='TEG', how='left')

# Define metrics and aggregation functions
metrics = {
    #'lowest Sc': 'Sc',
    'lowest GrossVP': 'GrossVP',
    'lowest NetVP': 'NetVP',
    'highest Stableford': 'Stableford'
}

# Initialize a dictionary to store outputs
output_dict = {}

# Loop over each metric and create the desired DataFrame
for metric_name, column in metrics.items():
    # Set up whether to find max or min for each metric
    if column == 'Stableford':
        best_values = df_merged.loc[df_merged.groupby('Player')[column].idxmax()]
    else:
        best_values = df_merged.loc[df_merged.groupby('Player')[column].idxmin()]

    # Select relevant columns and rename them
    output = best_values[['Player', column, 'TEG', 'Year']].rename(columns={column: metric_name})

    # Store the result in the dictionary
    output_dict[metric_name] = output
    
    # Print the output DataFrame
    print(f"Output for {metric_name}:")
    print(output, "\n")

In [ ]:
aggregate_data(data=all_data,aggregation_level='TEG')

In [ ]:
import pandas as pd
from IPython.display import Markdown

# Example DataFrame
data = {
    'Player': ['Alice', 'Bob', 'Charlie'],
    'Score': [72, 68, 74],
    'TEG': ['TEG1', 'TEG2', 'TEG1'],
    'Year': [2021, 2020, 2022],
    'Measure': ['Gross', 'Net', 'Stableford']
}

df = pd.DataFrame(data)

# Loop through DataFrame and display markdown text with dynamic content
for index, row in df.iterrows():
    # Define the markdown text using row values
    markdown_text = f"Best {row['Measure']}: **{row['Score']}** | {row['Player']} ({row['TEG']}, {row['Year']})"
    
    # Display the markdown text in Jupyter notebook
    display(Markdown(markdown_text))

In [ ]:
from utils import get_round_data, get_complete_teg_data, get_9_data

rds = get_round_data()
tegs = get_complete_teg_data()

#rds.head()

import pandas as pd

rank_field = 'Sc'
rank_ascending = True

def add_ranks(df, fields_to_rank=None, rank_ascending=None):
    # If fields_to_rank is not provided, use default list of fields
    if fields_to_rank is None:
        fields_to_rank = ['Sc', 'GrossVP', 'NetVP', 'Stableford']
    
    # Check if fields_to_rank is a string, convert to list if necessary
    if isinstance(fields_to_rank, str):
        fields_to_rank = [fields_to_rank]
    
    for field in fields_to_rank:
        # Determine default value for rank_ascending for each field
        if rank_ascending is None:
            rank_ascending = False if 'Stableford' in field else True
        
        # Rank within each Player's rounds
        df[f'Rank_within_player_{field}'] = df.groupby('Player')[field].rank(method='min', ascending=rank_ascending)
        
        # Rank across all rounds
        df[f'Rank_within_all_{field}'] = df[field].rank(method='min', ascending=rank_ascending)
    
    return df

#ranked_rds = add_ranks(rds,'Sc',rank_ascending=True)
#ranked_rds.to_clipboard()
# fields_to_rank = 'Sc', ''
# ranked_tegs = add_ranks(tegs,'Stableford')
# ranked_tegs.to_clipboard()

frontback_data = get_9_data()
ranked_9s = add_ranks(df = frontback_data)
ranked_9s.to_clipboard()
print('Copied to clipboard')

In [ ]:
for column in ranked_9s.columns:
    print(column)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# scatter = plt.scatter(df['Rank_within_all_Sc'], df['Sc'])
df = ranked_9s

df['rank'] = df['Rank_within_all_Sc']

# Get unique values from 'Pl' column
unique_pl = df['Pl'].unique()

# Create a color map
color_map = plt.cm.get_cmap('tab10')  # You can change 'tab10' to other colormaps
color_dict = {pl: color_map(i/len(unique_pl)) for i, pl in enumerate(unique_pl)}

# Map colors to your data
colors = df['Pl'].map(color_dict)

plt.figure(figsize=(10, 6))
for pl in unique_pl:
    mask = df['Pl'] == pl
    plt.scatter(df.loc[mask, 'rank'], df.loc[mask, 'Sc'], 
                c=[color_dict[pl]], label=pl)

plt.xlabel('Rank')
plt.ylabel('Mean (Sc)')
plt.title('Scatter Plot of Rank vs Mean (Sc), colored by Pl')
plt.legend()
plt.show()


import plotly.express as px

fig = px.scatter(df, x='rank', y='Sc', color='Pl', 
                 labels={'rank': 'Rank', 'Sc': 'Mean (Sc)', 'Pl': 'Pl'},
                 title='Scatter Plot of Rank vs Mean (Sc), colored by Pl')
fig.show()

In [ ]:
from utils import get_ranked_teg_data, get_ranked_round_data, get_ranked_frontback_data

import streamlit as st

st.cache_data.clear()

tegs_ranked = get_ranked_teg_data()
# for col in tegs_ranked.columns:
#     print(col)

def get_best(df, measure_to_use, player_level = False, top_n = 1):
    valid_measures = ['Sc', 'GrossVP', 'NetVP', 'Stableford']
    if measure_to_use not in valid_measures:
        error_message = f"Invalid measure: '{measure_to_use}'. Valid options are: {', '.join(valid_measures)}"

    if player_level is None:
        player_level = False

    if top_n is None:
        top_n = 1
    
    measure_fn = 'Rank_within_' + ('player' if player_level else 'all') + f'_{measure_to_use}' 

    #measure_fn
    return df[df[measure_fn] == top_n]

#get_best(get_ranked_round_data(), measure_to_use='Stableford')
get_best(tegs_ranked, measure_to_use='Stableford')

THIS BIT MAKES THE RECORDS

In [ ]:
tegs_ranked = get_ranked_teg_data()

def format_best_record(df, measure):
    # Get the best record(s)
    best_records = get_best(df, measure_to_use=measure, top_n=1)
    
    # Format the header based on the measure
    if measure == 'Sc':
        header = f"Best score: {int(best_records[measure].iloc[0])}"
    elif measure == 'GrossVP':
        header = f"Best gross: {int(best_records[measure].iloc[0]):+}"
    elif measure == 'NetVP':
        header = f"Best net: {int(best_records[measure].iloc[0]):+}"
    elif measure == 'Stableford':
        header = f"Best Stableford: {int(best_records[measure].iloc[0])}"
    
    # Format player info
    if len(best_records) == 1:
        player_info = f"| **{best_records['Player'].iloc[0]}** | {best_records['TEG'].iloc[0]} ({best_records['Year'].iloc[0]})"
        return f"{header} {player_info}"
    else:
        player_infos = [f"* **{row['Player']}** | {row['TEG']} ({row['Year']})" for _, row in best_records.iterrows()]
        return f"{header}\n" + "\n".join(player_infos)

# Usage
measures = ['Sc', 'GrossVP', 'NetVP', 'Stableford']
for measure in measures:
    print(format_best_record(tegs_ranked, measure))
    print()  # Add a blank line between measures

TESTING HOW TO PUT CHOSEN ROUNDS IN CONTEXT

In [12]:
from utils import get_best, get_ranked_teg_data, get_ranked_round_data, get_ranked_frontback_data,safe_ordinal

teg = 'TEG 15'
rd = 4

df = get_ranked_round_data()

def chosen_rd_context(ranked_rd_df, teg = 'TEG 15',rd = 4, measure = None):
    #@st.cache_data
    all_cnt = len(df)
    df['Pl_count'] = df.groupby('Pl')['Pl'].transform('count')
    chosen_rd = df[(df['TEG']==teg) & (df['Round'] == rd)]

    sort_ascending = measure != 'Stableford'
    chosen_rd = chosen_rd.sort_values(measure, ascending=sort_ascending)
    # chosen_rd['Pl rank'] = chosen_rd['Rank_within_player_' + measure].apply(safe_ordinal)
    # chosen_rd['All time rank'] = chosen_rd['Rank_within_all_' + measure].apply(safe_ordinal)
    #chosen_rd['Pl rank'] = f'{chosen_rd['Rank_within_player_' + measure].astype(int)} / {chosen_rd['Pl_count']}'
    chosen_rd['Pl rank'] = (chosen_rd['Rank_within_player_' + measure].astype(int).astype(str) + 
                        ' / ' + 
                        chosen_rd['Pl_count'].astype(str))
    chosen_rd['All time rank'] = (chosen_rd['Rank_within_all_' + measure].astype(int).astype(str) + 
                              ' / ' + 
                              str(all_cnt))
    chosen_rd_context = chosen_rd[['Player',measure,'Pl rank','All time rank']]
    chosen_rd_context[measure] = chosen_rd_context[measure].astype(int)
    return chosen_rd_context

print(chosen_rd_context(df,teg,rd,'Sc'))
print(chosen_rd_context(df,teg,rd,'GrossVP'))
print(chosen_rd_context(df,teg,rd,'NetVP'))
print(chosen_rd_context(df,teg,rd,'Stableford'))


def chosen_teg_context(ranked_teg_df, teg = 'TEG 15', measure = None):
    #@st.cache_data
    all_cnt = len(df)
    chosen_teg = df[(df['TEG']==teg)]

    sort_ascending = measure != 'Stableford'
    chosen_teg = chosen_teg.sort_values(measure, ascending=sort_ascending)
    # chosen_rd['Pl rank'] = chosen_rd['Rank_within_player_' + measure].apply(safe_ordinal)
    # chosen_rd['All time rank'] = chosen_rd['Rank_within_all_' + measure].apply(safe_ordinal)
    #chosen_rd['Pl rank'] = f'{chosen_rd['Rank_within_player_' + measure].astype(int)} / {chosen_rd['Pl_count']}'
    chosen_teg['Pl rank'] = (chosen_teg['Rank_within_player_' + measure].astype(int).astype(str) + 
                        ' / ' + 
                        chosen_teg['Pl_count'].astype(str))
    chosen_teg['All time rank'] = (chosen_teg['Rank_within_all_' + measure].astype(int).astype(str) + 
                              ' / ' + 
                              str(all_cnt))
    chosen_teg_context = chosen_teg[['Player',measure,'Pl rank','All time rank']]
    chosen_teg_context[measure] = chosen_rd_context[measure].astype(int)
    return chosen_teg_context

print(chosen_teg_context(df,teg,'Sc'))
print(chosen_teg_context(df,teg,'GrossVP'))
print(chosen_teg_context(df,teg,'NetVP'))
print(chosen_teg_context(df,teg,'Stableford'))


2024-10-13 20:22:56.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 20:22:56.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 20:22:56.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 20:22:56.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 20:22:56.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


             Player   Sc  Pl rank All time rank
279  Gregg WILLIAMS   91  16 / 59      80 / 315
280       Jon BAKER   94  39 / 59     130 / 315
282  Stuart NEUMANN   99  30 / 47     209 / 315
277      Alex BAKER  103  25 / 40     257 / 315
278    David MULLIN  104  59 / 59     262 / 315
281  John PATTERSON  105  19 / 24     266 / 315
             Player  GrossVP  Pl rank All time rank
279  Gregg WILLIAMS       20  16 / 59      87 / 315
280       Jon BAKER       23  40 / 59     144 / 315
282  Stuart NEUMANN       28  32 / 47     219 / 315
277      Alex BAKER       32  25 / 40     258 / 315
278    David MULLIN       33  59 / 59     265 / 315
281  John PATTERSON       34  20 / 24     272 / 315
             Player  NetVP  Pl rank All time rank
282  Stuart NEUMANN      1  18 / 47     143 / 315
279  Gregg WILLIAMS      2  35 / 59     159 / 315
280       Jon BAKER      7  50 / 59     247 / 315
277      Alex BAKER      8  34 / 40     256 / 315
281  John PATTERSON     10  19 / 24     280 / 315


C:\Users\JBA33\AppData\Local\Temp\ipykernel_27124\2071994657.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen_rd_context[measure] = chosen_rd_context[measure].astype(int)
C:\Users\JBA33\AppData\Local\Temp\ipykernel_27124\2071994657.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen_rd_context[measure] = chosen_rd_context[measure].astype(int)
C:\Users\JBA33\AppData\Local\Temp\ipykernel_27124\2071994657.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a